In [ ]:
!git clone https://github.com/varungupta31/DeIT-CIFAR.git


In [3]:
!pwd

/content


#Imported Important Libraries

In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import os
import torchvision.models as models
from sklearn.metrics import *
from sklearn.metrics import top_k_accuracy_score
import numpy as np
from vit import VisionTransformer
from deit import DataEfficientImageTransformer as DEIT
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import yaml
# import argparse
# from imagenet32_dataloader import ImageNet32
torch.manual_seed(123)

## Function For Get Inference From Token

In [2]:
def getInferenceHardToken(model, loader,  device):

    model.eval()
    cls_token_acc1_log = 0
    distill_token_acc1_log = 0
    cls_distill_token_acc1_log = 0

    cls_token_prec = 0
    cls_token_rec = 0
    cls_token_fscore = 0
    
    distill_token_prec = 0
    distill_token_rec = 0
    distill_token_fscore = 0
    
    cls_distill_token_prec = 0
    cls_distill_token_rec = 0
    cls_distill_token_fscore = 0

    cls_token_precision = []
    cls_token_recall = []
    cls_token_f1Score = []
    
    distill_token_precision = []
    distill_token_recall = []
    distill_token_f1Score = []
    
    cls_distill_token_precision = []
    cls_distill_token_recall = []
    cls_distill_token_f1Score = []
    
    cls_token_acc3_log = 0
    distill_token_acc3_log = 0
    cls_distill_token_acc3_log = 0

    cls_token_acc5_log = 0
    distill_token_acc5_log = 0
    cls_distill_token_acc5_log = 0

    all_gt = []
    all_ground = [] 
    with torch.no_grad():
        for batch_index , (data, gt) in enumerate(loader):
            data = data.to(device)
            gt = gt.to(device)
            scores_cls_token, scores_distill_token = model(data)
            scores_cls_distill_token = scores_cls_token + scores_distill_token

            scores_cls_token =F.softmax(scores_cls_token, dim = 1)
            scores_distill_token = F.softmax(scores_distill_token, dim = 1)
            scores_cls_distill_token = F.softmax(scores_cls_distill_token, dim = 1)

            scores_cls_token = scores_cls_token.cpu().detach().numpy()
            scores_distill_token = scores_distill_token.cpu().detach().numpy()
            scores_cls_distill_token = scores_cls_distill_token.cpu().detach().numpy()

            gt = gt.cpu().detach().numpy()
            
            labels = np.arange(0,100)


            # if acc_mode == "top1":
            cls_token_acc1 = top_k_accuracy_score(gt,scores_cls_token, k=1, labels = labels)*100
            distill_token_acc1 = top_k_accuracy_score(gt, scores_distill_token, k = 1, labels = labels)*100
            cls_distill_token_acc1 = top_k_accuracy_score(gt, scores_cls_distill_token,k = 1, labels = labels)*100

            cls_token_prec += precision_score(gt, np.argmax(scores_cls_token, axis=-1), average= "macro", zero_division=0)*100
            cls_token_rec += recall_score(gt, np.argmax(scores_cls_token, axis=-1), average= "macro",zero_division = 0)*100
            cls_token_fscore += f1_score(gt, np.argmax(scores_cls_token, axis=-1), average= "macro",zero_division = 0)*100

            distill_token_prec += precision_score(gt, np.argmax(scores_distill_token, axis=-1), average= "macro", zero_division=0)*100
            distill_token_rec += recall_score(gt, np.argmax(scores_distill_token, axis=-1), average= "macro",zero_division = 0)*100
            distill_token_fscore += f1_score(gt, np.argmax(scores_distill_token, axis=-1), average= "macro",zero_division = 0)*100

            cls_distill_token_prec += precision_score(gt, np.argmax(scores_cls_distill_token, axis=-1), average= "macro", zero_division=0)*100
            cls_distill_token_rec += recall_score(gt, np.argmax(scores_cls_distill_token, axis=-1), average= "macro",zero_division = 0)*100
            cls_distill_token_fscore += f1_score(gt, np.argmax(scores_cls_distill_token, axis=-1), average= "macro",zero_division = 0)*100


            cls_token_acc3 = top_k_accuracy_score(gt,scores_cls_token, k=3, labels = labels)*100
            distill_token_acc3 = top_k_accuracy_score(gt, scores_distill_token, k = 3, labels = labels)*100
            cls_distill_token_acc3 = top_k_accuracy_score(gt, scores_cls_distill_token,k = 3, labels = labels)*100


            cls_token_acc5 = top_k_accuracy_score(gt,scores_cls_token, k=5, labels = labels)*100
            distill_token_acc5 = top_k_accuracy_score(gt, scores_distill_token, k = 5, labels = labels)*100
            cls_distill_token_acc5 = top_k_accuracy_score(gt, scores_cls_distill_token,k = 5, labels = labels)*100

           
            
            batch_cls_token_precision= cls_token_prec/(batch_index+1)
            batch_cls_token_recall= cls_token_rec/(batch_index+1)
            batch_cls_token_fscore= cls_token_fscore/(batch_index+1)

            batch_distill_token_precision= distill_token_prec/(batch_index+1)
            batch_distill_token_recall= distill_token_rec/(batch_index+1)
            batch_distill_token_fscore= distill_token_fscore/(batch_index+1)

            batch_cls_distill_token_precision= cls_distill_token_prec/(batch_index+1)
            batch_cls_distill_token_recall= cls_distill_token_rec/(batch_index+1)
            batch_cls_distill_token_fscore= cls_distill_token_fscore/(batch_index+1)


            cls_token_precision.append(batch_cls_token_precision)
            cls_token_recall.append(batch_cls_token_recall)
            cls_token_f1Score.append(batch_cls_token_fscore)
            
            distill_token_precision.append(batch_distill_token_precision)
            distill_token_recall.append(batch_distill_token_recall)
            distill_token_f1Score.append(batch_distill_token_fscore)
            
            cls_distill_token_precision.append(batch_cls_distill_token_precision)
            cls_distill_token_recall.append(batch_cls_distill_token_recall)
            cls_distill_token_f1Score.append(batch_cls_distill_token_fscore)


            cls_token_acc1_log += cls_token_acc1
            distill_token_acc1_log += distill_token_acc1
            cls_distill_token_acc1_log += cls_distill_token_acc1 

            cls_token_acc3_log += cls_token_acc3
            distill_token_acc3_log += distill_token_acc3
            cls_distill_token_acc3_log += cls_distill_token_acc3

            cls_token_acc5_log += cls_token_acc5
            distill_token_acc5_log += distill_token_acc5
            cls_distill_token_acc5_log += cls_distill_token_acc5

        cls_token_precision = np.mean(cls_token_precision)
        cls_token_recall = np.mean(cls_token_recall)
        cls_token_f1Score = np.mean(cls_token_f1Score)
        
        distill_token_precision = np.mean(distill_token_precision)
        distill_token_recall = np.mean(distill_token_recall)
        distill_token_f1Score = np.mean(distill_token_f1Score)
        
        cls_distill_token_precision = np.mean(cls_distill_token_precision)
        cls_distill_token_recall = np.mean(cls_distill_token_recall)
        cls_distill_token_f1Score = np.mean(cls_distill_token_f1Score)

        return (cls_token_acc1_log/(batch_index+1),
                    distill_token_acc1_log/(batch_index+1),
                    cls_distill_token_acc1_log/(batch_index+1),
                    cls_token_precision, cls_token_recall, 
                    distill_token_precision, distill_token_recall,
                    cls_distill_token_precision, cls_distill_token_recall,
                    cls_token_f1Score, distill_token_f1Score, cls_distill_token_f1Score,
                    cls_token_acc3_log/(batch_index+1),
                    distill_token_acc3_log/(batch_index+1),
                    cls_distill_token_acc3_log/(batch_index+1),
                    cls_token_acc5_log/(batch_index+1),
                    distill_token_acc5_log/(batch_index+1),
                    cls_distill_token_acc5_log/(batch_index+1)
                    )

## Function For Get Inference From Without Token

In [3]:
def getInference(model, loader,  device):
    model.eval()
    out_acc1 = 0
    out_acc3 = 0
    out_acc5 = 0
    out_precision = []
    out_recall = []
    out_f1Score = []
    f1Score = 0
    prec = 0
    rec = 0
    for batch_index ,(data, gt) in enumerate(loader):
        data = data.to(device)
        gt = gt.to(device)
        scores = model(data)
        scores = F.softmax(scores, dim = 1)
        scores = scores.cpu().detach().numpy()
        gt = gt.cpu().detach().numpy()
 
        labels = np.arange(0,100)


        acc1 = top_k_accuracy_score(gt,scores, k=1, labels = labels)*100
        acc3 = top_k_accuracy_score(gt,scores, k=3, labels = labels)*100
        acc5 = top_k_accuracy_score(gt,scores, k=5, labels = labels)*100

        out_acc1 += acc1
        out_acc3 += acc3
        out_acc5 += acc5
        

        prec += precision_score(gt, np.argmax(scores, axis=-1), average = "macro", zero_division = 0)*100
        rec += recall_score(gt, np.argmax(scores, axis=-1),average  = "macro",zero_division = 0)*100
        f1Score += f1_score(gt, np.argmax(scores, axis=-1), average = "macro",zero_division = 0)*100
        
        batch_prec = prec/(batch_index+1)
        batch_rec = rec/(batch_index+1)
        batch_f1Score = f1Score/(batch_index+1)
      
        out_precision.append(batch_prec)
        out_recall.append(batch_rec)
        out_f1Score.append(batch_f1Score)
    
    out_precision = np.mean(out_precision)
    out_recall = np.mean(out_recall)
    out_f1Score = np.mean(out_f1Score)
    return (out_acc1/(batch_index+1),
                out_acc3/(batch_index+1),
                out_acc5/(batch_index+1),
                out_precision,out_recall,
                out_f1Score)

In [4]:
def doInference(loader, mode = "distilled_token", model= None,  device="cuda"):
    if mode == "distilled_token":

        final_model = model
        
        loader = loader
        device = device
        

        
        if final_model == None:
            print("No model is provided!!")
        else:
            clsTokenAcc1, distillTokenAcc1, clsDistillTokenAcc1, cls_token_precision, cls_token_recall,distill_token_precision, distill_token_recall,cls_distill_token_precision, cls_distill_token_recall, cls_token_f1Score, distill_token_f1Score, cls_distill_token_f1Score,clsTokenAcc3, distillTokenAcc3, clsDistillTokenAcc3,clsTokenAcc5, distillTokenAcc5, clsDistillTokenAcc5= getInferenceHardToken(final_model,loader, device)
            print("Final Top-1 Accuracy")
     
            print(f"cls_token_acc:{clsTokenAcc1}\n distill_token_acc:{distillTokenAcc1}\n cls_distill_token_acc:{clsDistillTokenAcc1}\n"
      f"prec_cls_token: {cls_token_precision}\n recall_cls_token: {cls_token_recall}\n cls_token_f1Score: {cls_token_f1Score}"
      f"prec_distill_token:{distill_token_precision}\n recall_distill_token: {distill_token_recall}\n distill_token_f1Score : {distill_token_f1Score} "
             f"prec_class_dist_token:{cls_distill_token_precision}\n recall_class_distill_token:{cls_distill_token_recall}\n cls_distill_token_f1Score : {cls_distill_token_f1Score}")
     

            print("************************************")

            print("Final Top-3 Accuracy")
            print(f"cls_token_acc:{clsTokenAcc3}\n distill_token_acc:{distillTokenAcc3}\n cls_distill_token_acc:{clsDistillTokenAcc3}\n")
            print("************************************")

            print("Final Top-5 Accuracy")
            print(f"cls_token_acc:{clsTokenAcc5}\n distill_token_acc:{distillTokenAcc5}\n cls_distill_token_acc:{clsDistillTokenAcc5}\n")
     

        

    else:
        final_model = model
        # print("entering.....")
        loader = loader
        device = device
        
        if final_model == None:
            print("No model is provided!!")
        else:
            
            acc_1,acc_3,acc_5,out_precision,out_recall,out_f1Score = getInference(final_model,loader,  device)
            print("Final Accuracy.....")
            print(f"acc@1:{acc_1}\t acc@3 :{acc_3}\t acc@5 :{acc_5} precision:{out_precision}\t recall:{out_recall}\t f1_score:{out_f1Score}")

In [ ]:
import torchvision.transforms as transforms
transforms = transforms.Compose([transforms.ToTensor(), 
                            transforms.Normalize((0.2675, 0.2565, 0.2761),(0.5071, 0.4867, 0.4408))])
#load the test set
testset = datasets.CIFAR100(root = '/', train = False, transform = transforms, download = True)
batch_size = 1024
test_dataloaders = DataLoader(testset, batch_size = batch_size, shuffle = "False", num_workers = 4)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
custom_config = {
    "img_size": 32,
    "in_chans": 3,
    "patch_size": 16,
    "embed_dim": 768,
    "depth": 12,
    "n_heads": 12,
    "qkv_bias": True,
    "mlp_ratio": 4,
}

##Initializing the Model

In [8]:
model = DEIT(**custom_config)

## Loading The Model 

In [9]:
#loading the  model
saved_model_path = "/content/drive/MyDrive/smai_project_trained_files/vit_b_reg16gf_hard_dist_token_24"
model.load_state_dict(torch.load(saved_model_path))
model.to(device)

DataEfficientImageTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (cls_head): Linear(in_features=768, out_f

In [10]:
total_num_param = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters for:{total_num_param}")

Total number of parameters for:85806536


In [11]:
doInference(mode = "distilled_token" , model= model, loader= test_dataloaders)

Final Top-1 Accuracy
cls_token_acc:33.63958864795918
 distill_token_acc:33.09809470663265
 cls_distill_token_acc:33.929567920918366
prec_cls_token: 34.68550067225836
 recall_cls_token: 34.027773573834544
 cls_token_f1Score: 33.04515976814497prec_distill_token:33.92327389693391
 recall_distill_token: 33.15488635350676
 distill_token_f1Score : 32.10729739230323 prec_class_dist_token:34.69782749425314
 recall_class_distill_token:33.86154537588611
 cls_distill_token_f1Score : 32.950956906753014
************************************
Final Top-3 Accuracy
cls_token_acc:52.00972576530612
 distill_token_acc:52.14704241071429
 cls_distill_token_acc:52.95459980867347

************************************
Final Top-5 Accuracy
cls_token_acc:60.48290019132653
 distill_token_acc:60.92534279336735
 cls_distill_token_acc:61.15672831632653

